In [60]:
import spotipy
import json
import webbrowser
import pandas as pd

In [61]:
# Importing library for spotify
import urllib.request
import spotipy.util as util

In [62]:
# Importing geolocation library and getting user's location
import geocoder
userGeolocation = geocoder.ip('me')

In [63]:
# open file with keys and set the path to your credentials JSON file
credentials = "SpotifyTokens.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)
    
# read the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]
trafficKey = api_tokens["trafficKey"]

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-currently-playing'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [64]:
# Creating a variable to store authorised spotipy function
sp = spotipy.Spotify(auth=token)

In [65]:
# Passing my traffic key to request traffic data from London
request = urllib.request.Request("http://dev.virtualearth.net/REST/v1/Traffic/Incidents/51,%20-0.489,%2051.686,%200.236?key=" + trafficKey)
response = urllib.request.urlopen(request)

In [66]:
#Transforming the data to json
fullTrafficData = json.loads(response.read())

In [67]:
amountTrafficCases = fullTrafficData["resourceSets"][0]['estimatedTotal']
trafficData = fullTrafficData["resourceSets"][0]['resources']

In [68]:
# Making a boolean to check if song is being played or not
playingMusic = False

my_current_track = sp.current_user_playing_track()
#print(current_track['item']['name'])

# my_current_track only seems to be of type dict when a song is being played
# So if its a dictionary, music is being played
if type(my_current_track) is dict:
    playingMusic = True

# If no song is being played, print "no song being played", otherwise print the artis nam and song being played
if playingMusic == False:
    print ("No song being played")
else:
    print(f"Listening to {my_current_track['item']['name']} by {my_current_track['item']['album']['artists'][0]['name']}")


No song being played


In [59]:
track_results = sp.search(q='stuck', type='track', limit=50)
tracks = track_results['tracks']['items']

# URI = uniform resource identifier
track_selection_list = []
track_art = []

for song in tracks:
    track_selection_list.append(song['uri'])
    track_art.append(song['album']['images'][0]['url'])

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'stuck', 'limit': 50, 'offset': 0, 'type': 'track', 'market': None} returned 401 due to The access token expired


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?q=stuck&limit=50&offset=0&type=track:
 The access token expired, reason: None

In [47]:
newTrafficData = {}
for x in trafficData:
    newTrafficData = x
    
print(newTrafficData['description'])

Incident


In [30]:
congestionDescription = []
for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
        congestionDescription += x['description'].split(".")
#print(congestionDescription)

In [32]:
# Not all entries seem to have a 'title' so its creating problems
coordAndSteet = []
for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
#         print(' Coordinate: ' + str(x['point']['coordinates']) + ' Street: ' + str(x['title']))
        coordAndSteet.append(' Coordinate: ' + str(x['point']['coordinates']) + ' Street: ' + x['title'])
print(coordAndSteet)

KeyError: 'title'

In [33]:
severeCongestionCoordinates = []
for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
        severeCongestionCoordinates.append(x['point']['coordinates'])
print(severeCongestionCoordinates)

[[51.524683, -0.084062], [51.519288, -0.102981], [51.543258, -0.129254], [51.359465, 0.09509], [51.41115546, 0.18190742], [51.40643477, 0.16907573], [51.621702, -0.421245], [51.624867, -0.423201], [51.582405, 0.042799], [51.555287, -0.129543], [51.195405, -0.396482], [51.194972, -0.397374], [51.553754, -0.130749], [51.60204, -0.396946], [51.601627, -0.398333], [51.511648, -0.083773], [51.513212, -0.089064], [51.513367, -0.088893], [51.424842, -0.420544], [51.424889, -0.419392], [51.525657, -0.166352], [51.524772, -0.168342], [51.493171, -0.063673], [51.549647, -0.152157], [51.487175, 0.004331], [51.359405, 0.0974], [51.602928, -0.117946], [51.534376, -0.063627], [51.535006, -0.066744], [51.217268, 0.226358], [51.217706, 0.221753], [51.598942, -0.290771], [51.50867, -0.199774], [51.421288, -0.152628], [51.422453, -0.155385], [51.423344, -0.145726], [51.422815, -0.145107], [51.421707, -0.147583], [51.430805, -0.173012], [51.548251, -0.12765], [51.526767, -0.091161], [51.53725, 0.026491],

In [34]:
severeCongestionStreets = []

for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
        severeCongestionStreets.append(x['title'])
        #print(x['title'])
print(severeCongestionStreets)
        
## I think theres an error in the data where one of the current lines does not have a title so its complaining if i print it outside of th loop

KeyError: 'title'

In [35]:
# Creating a playlist for each congestion coordinate

for x in range(len(severeCongestionCoordinates)):
    playlistCoordinateString = str(severeCongestionCoordinates[x])
    
    my_playlists = sp.user_playlist_create(user=username, name=severeCongestionCoordinates, public=True,
                                      description="Playlist made where a traffic jam is")
    results = sp.user_playlist_add_tracks(username, my_playlists['id'], track_selection_list)

print(my_playlists)

HTTP Error for POST to https://api.spotify.com/v1/users/pinguinarey/playlists with Params: {} returned 400 due to Error parsing JSON.


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/users/pinguinarey/playlists:
 Error parsing JSON., reason: None

In [46]:
#Hardcoding a coordinate that is in traffic
testyUserInTraffic = [51.518559, -0.218466]

#Creating booleans to track if the user is in traffic or not
hardcodedUserStatusInTraffic = False
myRealStatusInTraffic = False

# for in range loop with a hardcoded lat and long to equate to true
# probs need to hardcode the value again in the future if i want to test again since live data might change

for x in range(len(severeCongestionCoordinates)):
    if testyUserInTraffic == severeCongestionCoordinates[x]:
        hardcodedUserStatusInTraffic = True


# for in range loop with my real geolocation
for x in range(len(severeCongestionCoordinates)):
    if userGeolocation.latlng == severeCongestionCoordinates[x]:
        myRealStatusInTraffic = True


True
False


In [94]:
# Creating a playlist with the street names instead of the coordinates

for x in range(len(severeCongestionCoordinates)):
    playlistCoordinateString = str(severeCongestionCoordinates[x])
    
    my_playlists = sp.user_playlist_create(user=username, name=playlistCoordinateString, public=True,
                                      description="Playlist made where a traffic jam is")
    results = sp.user_playlist_add_tracks(username, my_playlists['id'], track_selection_list)

print(results)

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [40]:
print(my_playlists)

{'collaborative': False, 'description': 'Playlist made where a traffic jam is', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4gdAeXWdUKyW5NFI9t24C2'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/4gdAeXWdUKyW5NFI9t24C2', 'id': '4gdAeXWdUKyW5NFI9t24C2', 'images': [], 'name': '[51.525313, -0.260287]', 'owner': {'display_name': 'Rocio', 'external_urls': {'spotify': 'https://open.spotify.com/user/pinguinarey'}, 'href': 'https://api.spotify.com/v1/users/pinguinarey', 'id': 'pinguinarey', 'type': 'user', 'uri': 'spotify:user:pinguinarey'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MSw0YTg5YmIzOGE1NmM5ZThjZTIyMDA1NzFlZjViYTJmN2NlYzllZGEw', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/4gdAeXWdUKyW5NFI9t24C2/tracks', 'items': [], 'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'total': 0}, 'type': 'playlist', 'uri': 'spotify:playlist:4gdAeXWdUKyW5NFI9t24C2'}


In [41]:
# sp.start_playback(deviceID, None, track_selection_list)
webbrowser.open(my_playlists['external_urls']['spotify'])

True